In [1]:
import pandas as pd
import numpy
import datetime
import random
import json
from models import Reciepts, RecieptItem, Recipe, BOH_Item, Server


In [2]:
BOH_ITEM_COUNT = 1000
RECIPES_COUNT = 50
RECIEPTS_COUNT = 10000
RECIEPT_ITEM_COUNT = 20
SERVERS_COUNT = 200

In [3]:
boh_items_df = pd.DataFrame({
    'id': [x+1 for x in range(BOH_ITEM_COUNT)],
    'name': [f'item_name_{x+1}' for x in range(BOH_ITEM_COUNT)],
    'price_per_unit': [round(random.uniform(0,10), 2) for x in range(BOH_ITEM_COUNT)]
})
boh_items_df.head()

,id,name,price_per_unit
0,1,item_name_1,5.18
1,2,item_name_2,2.16
2,3,item_name_3,2.70
3,4,item_name_4,2.85
4,5,item_name_5,4.73


In [4]:
def recipe_total_price(recipe):
    total = 0
    for x in recipe:
        total += boh_items_df[boh_items_df['id'] == int(x)]['price_per_unit'].tolist()[0]
    return total


recipes_df = pd.DataFrame({
    'id': [x+1 for x in range(RECIPES_COUNT)],
    'recipe_name': [f'recipe_{x+1}' for x in range(RECIPES_COUNT)],
    'recipe': [str([random.randint(1,RECIPES_COUNT) for x in range(random.randint(2,RECIPES_COUNT))])[1:-1] for x in range(RECIPES_COUNT)]
})

recipes_df['price_to_create_dish'] = [recipe_total_price(recipes_df['recipe'].tolist()[x].split(',')) for x in range(RECIPES_COUNT)]

recipes_df.head()

,id,recipe_name,recipe,price_to_create_dish
0,1,recipe_1,"42, 28, 30, 38, 3, 50, 39, 23, 21, 47, 33, 20,...",147.05
1,2,recipe_2,"14, 27, 44, 19, 12, 36, 34, 13, 50, 29, 26, 37...",72.51
2,3,recipe_3,"17, 8, 23, 8, 9",28.81
3,4,recipe_4,"2, 26, 8, 6, 31, 7, 1, 41",38.78
4,5,recipe_5,"11, 10, 7, 17, 9, 6, 41, 42, 32, 37, 21, 20, 3...",179.47


In [5]:
def gross_sum(items: list):
    total = 0
    for x in items:
        total += recipes_df[recipes_df['id'] == int(x)]['price_to_create_dish'].tolist()[0]
    return total


reciepts_df = pd.DataFrame({
    'id': [x+1 for x in range(RECIEPTS_COUNT)],
    'items': [str([recipes_df['id'].tolist()[random.randint(0,len(recipes_df)-1)] for y in range(random.randint(1,RECIEPT_ITEM_COUNT))])[1:-1] for x in range(RECIEPTS_COUNT)],
    'server_id': [random.randint(0,SERVERS_COUNT) for x in range(RECIEPTS_COUNT)],
})

reciepts_df['gross'] = [gross_sum(reciepts_df['items'].tolist()[x].split(',')) for x in range(RECIEPTS_COUNT)]
reciepts_df['tax'] = reciepts_df['gross'] * 0.0875
reciepts_df['total'] = reciepts_df['tax'] + reciepts_df['gross']
"""
'tax': [],
'total': [],
'time': [],
"""

reciepts_df.head()

,id,items,server_id,gross,tax,total
0,1,"11, 40, 30, 9, 22, 45",51,874.82,76.546750,951.366750
1,2,"13, 27, 40, 35, 1, 32, 22, 22, 44, 36, 12, 34,...",141,2560.94,224.082250,2785.022250
2,3,"16, 6, 27, 48, 49, 3, 5, 20, 47, 40, 22, 26, 3...",169,2413.97,211.222375,2625.192375
3,4,"33, 48, 12, 16, 25, 38, 11, 13, 47, 22, 41, 18",195,1594.17,139.489875,1733.659875
4,5,"10, 34, 28, 11, 32, 45, 50, 48, 18, 19, 39, 6,...",194,2729.80,238.857500,2968.657500


In [6]:
def random_date():
    return numpy.datetime64('2005-01-01') - random.randint(1, 65*365)


def random_hire_date():
    return numpy.datetime64('2023-01-01') - random.randint(1, 10*365)

server_df = pd.DataFrame({
    'id': [x+1 for x in range(SERVERS_COUNT)],
    'name': [f'server_name_{x+1}' for x in range(SERVERS_COUNT)],
})
server_df['born'] = [random_date() for x in range(SERVERS_COUNT)]
server_df['hire_date'] = [random_hire_date() for x in server_df['born'].tolist()]

server_df.head()

,id,name,born,hire_date
0,1,server_name_1,1988-05-30,2014-03-16
1,2,server_name_2,1990-01-01,2020-01-06
2,3,server_name_3,1988-02-14,2020-12-24
3,4,server_name_4,1959-09-28,2022-03-10
4,5,server_name_5,1979-11-04,2015-09-26


In [8]:
import os
if 'resources' not in os.listdir():
    os.mkdir('resources')
boh_items_df.to_csv('resources/boh_items.csv', index=False)
recipes_df.to_csv('resources/recipes.csv', index=False)
reciepts_df.to_csv('resources/reciepts.csv', index=False)
server_df.to_csv('resources/servers.csv', index=False)